# Projet 7 - Réalisez une analyse de sentiments grâce au Deep Learning

> 🎓 OpenClassrooms • Parcours [AI Engineer](https://openclassrooms.com/fr/paths/795-ai-engineer) | 👋 *Étudiant* : [David Scanu](https://www.linkedin.com/in/davidscanu14/)

## Partie 4 : Analyse de Sentiment de Tweets avec BERT

Dans ce notebook, nous explorons l'analyse de sentiment de tweets en utilisant **BERT (Bidirectional Encoder Representations from Transformers)**, un modèle de deep learning à la pointe de la technologie pour le traitement du langage naturel. Le dataset **Sentiment140** contient 1,6 million de tweets étiquetés (positifs ou négatifs), ce qui en fait une excellente ressource pour entraîner notre modèle. Contrairement aux approches traditionnelles qui nécessitent une ingénierie de caractéristiques complexe, **BERT comprend le contexte bidirectionnel des mots**, ce qui est particulièrement utile pour capturer les subtilités linguistiques des tweets comme l'ironie, les abréviations et les expressions idiomatiques propres aux réseaux sociaux.

## Étapes principales du projet :

1. **Préparation des données**
   - Chargement du dataset Sentiment140
   - Prétraitement spécifique aux tweets (URLs, mentions, hashtags)
   - Division en ensembles d'entraînement/validation/test

2. **Création et entraînement du modèle BERT**
   - Initialisation du modèle pré-entraîné BERT
   - Configuration des hyperparamètres d'entraînement
   - Entraînement avec suivi des métriques (accuracy, loss)

3. **Évaluation des performances**
   - Calcul des métriques (précision, rappel, F1-score, AUC)
   - Visualisation de la matrice de confusion et de la courbe ROC
   - Analyse détaillée des erreurs de classification

4. **Intégration MLflow**
   - Suivi des expériences et des métriques
   - Sauvegarde des artefacts (modèle, tokenizer, graphiques)
   - Documentation des hyperparamètres pour reproductibilité

5. **Déploiement et utilisation**
   - Chargement du modèle sauvegardé
   - Création d'une fonction de prédiction pour nouveaux tweets
   - Tests sur un ensemble diversifié de tweets

Ce notebook présente une solution complète et industrialisée pour l'analyse de sentiment, de l'entraînement à l'évaluation jusqu'au déploiement, en suivant les meilleures pratiques de MLOps avec MLflow.

## 📝 Contexte

Dans le cadre de ma formation d'AI Engineer chez OpenClassrooms, ce projet s'inscrit dans un scénario professionnel où j'interviens en tant qu'ingénieur IA chez MIC (Marketing Intelligence Consulting), entreprise de conseil spécialisée en marketing digital.

Notre client, Air Paradis (compagnie aérienne), souhaite **anticiper les bad buzz sur les réseaux sociaux**. La mission consiste à développer un produit IA permettant de **prédire le sentiment associé à un tweet**, afin d'améliorer la gestion de sa réputation en ligne.

## ⚡ Mission

> Développer un modèle d'IA permettant de prédire le sentiment associé à un tweet.

Créer un prototype fonctionnel d'un modèle d'**analyse de sentiments pour tweets** selon trois approches différentes :

1. **Modèle sur mesure simple** : Approche classique (régression logistique) pour une prédiction rapide
2. **Modèle sur mesure avancé** : Utilisation de réseaux de neurones profonds avec différents word embeddings
3. **Modèle avancé BERT** : Exploration de l'apport en performance d'un modèle BERT

Cette mission implique également la mise en œuvre d'une démarche MLOps complète :
- Utilisation de **MLFlow** pour le tracking des expérimentations et le stockage des modèles
- Création d'un pipeline de déploiement continu (Git + Github + plateforme Cloud)
- Intégration de tests unitaires automatisés
- Mise en place d'un suivi de performance en production via Azure Application Insight

## Importation des bibliothèques

In [ ]:
import torch
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA disponible: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"Appareil CUDA: {torch.cuda.get_device_name(0)}")

In [ ]:
import os
import re
import pandas as pd
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
from torch.cuda.amp import autocast, GradScaler
from torch.optim import AdamW 
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification, get_linear_schedule_with_warmup
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import confusion_matrix, classification_report, roc_curve, auc
import matplotlib
matplotlib.use('Agg')  # Backend qui fonctionne dans tous les environnements
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
import pickle
import zipfile
import requests
import json
import mlflow
from mlflow.models import infer_signature
from mlflow import MlflowClient
from dotenv import load_dotenv

# Charger les variables d'environnement depuis le fichier .env
load_dotenv()

# Configuration de MLflow avec les variables d'environnement
mlflow_tracking_uri = os.getenv("MLFLOW_TRACKING_URI")
aws_access_key_id = os.getenv("AWS_ACCESS_KEY_ID")
aws_secret_access_key = os.getenv("AWS_SECRET_ACCESS_KEY")

# Configuration explicite de MLflow
mlflow.set_tracking_uri(mlflow_tracking_uri)
print(f"MLflow Tracking URI: {mlflow_tracking_uri}")

# Configuration explicite des identifiants AWS
os.environ["AWS_ACCESS_KEY_ID"] = aws_access_key_id
os.environ["AWS_SECRET_ACCESS_KEY"] = aws_secret_access_key
print("Identifiants AWS configurés")

# Créer l'expérience MLflow
mlflow.set_experiment("OC Projet 7")


## Jeu de données 

In [ ]:
%%time 

# Define the URL and the local file path
url = "https://s3-eu-west-1.amazonaws.com/static.oc-static.com/prod/courses/files/AI+Engineer/Project+7%C2%A0-+D%C3%A9tectez+les+Bad+Buzz+gr%C3%A2ce+au+Deep+Learning/sentiment140.zip"
local_zip_path = "./content/data/sentiment140.zip"
extract_path = "./content/data"

if not os.path.exists(extract_path):

    # Create the directory if it doesn't exist
    os.makedirs(extract_path, exist_ok=True)

    # Download the zip file
    response = requests.get(url)
    with open(local_zip_path, 'wb') as file:
        file.write(response.content)

    # Extract the contents of the zip file
    with zipfile.ZipFile(local_zip_path, 'r') as zip_ref:
        zip_ref.extractall(extract_path)

    # Delete the zip file
    os.remove(local_zip_path)

In [ ]:
# Définir le chemin vers le fichier CSV
csv_file_path = './content/data/training.1600000.processed.noemoticon.csv'

## Prétraitement des tweets

In [ ]:
# PRÉTRAITEMENT DES TWEETS
def preprocess_tweet_for_bert(tweet):
    """
    Prétraite un tweet pour l'entraînement BERT en conservant la structure naturelle
    du langage mais en normalisant certains éléments spécifiques aux réseaux sociaux.
    """
    # Vérifier si le tweet est une chaîne de caractères
    if not isinstance(tweet, str):
        return ""
    
    # Remplacer les URLs par un token spécial
    tweet = re.sub(r'https?://\S+|www\.\S+', '[URL]', tweet)
    
    # Remplacer les mentions par un token spécial
    tweet = re.sub(r'@\w+', '[USER]', tweet)
    
    # Normaliser les hashtags (conserver le hashtag comme entité)
    tweet = re.sub(r'#(\w+)', r'#\1', tweet)
    
    # Normaliser les espaces multiples
    tweet = re.sub(r'\s+', ' ', tweet)
    
    # Supprimer les caractères non imprimables et certains caractères spéciaux inutiles
    tweet = re.sub(r'[^\x20-\x7E]', '', tweet)
    
    # Nettoyer les espaces en début et fin
    tweet = tweet.strip()
    
    return tweet

## Préparations des données 

In [ ]:
# PRÉPARATION DES DONNÉES
def prepare_data(df, sample_size=50000, random_state=42):
    """
    Préparation des données pour l'entraînement BERT
    """
    # Remappage des labels (0=négatif, 4=positif) vers (0=négatif, 1=positif)
    df['target'] = df['target'].replace({0: 0, 4: 1})
    
    # Prétraitement des tweets
    print("Prétraitement des tweets...")
    df['processed_text'] = df['text'].apply(preprocess_tweet_for_bert)
    
    # Sélection d'un échantillon pour l'entraînement
    sample_data = df.sample(n=sample_size, random_state=random_state)
    
    # Division train/val/test
    train_val_texts, test_texts, train_val_labels, test_labels = train_test_split(
        sample_data['processed_text'].values, 
        sample_data['target'].values,
        test_size=0.2,
        random_state=random_state
    )
    
    train_texts, val_texts, train_labels, val_labels = train_test_split(
        train_val_texts,
        train_val_labels,
        test_size=0.2,
        random_state=random_state
    )
    
    return {
        'train': {'texts': train_texts, 'labels': train_labels},
        'val': {'texts': val_texts, 'labels': val_labels},
        'test': {'texts': test_texts, 'labels': test_labels}
    }

In [ ]:
# CRÉATION D'UN DATASET PYTORCH
class TweetDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_length=128):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length
        
    def __len__(self):
        return len(self.texts)
    
    def __getitem__(self, idx):
        text = str(self.texts[idx])
        label = self.labels[idx]
        
        # Tokenisation du texte
        encoding = self.tokenizer(
            text,
            add_special_tokens=True,
            max_length=self.max_length,
            truncation=True,
            padding='max_length',
            return_tensors='pt'
        )
        
        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'label': torch.tensor(label, dtype=torch.long)
        }

## Fonctions d'entraînement

In [ ]:
# FONCTION D'ENTRAÎNEMENT
def train_model(model, train_loader, val_loader, test_loader, device, epochs=3, gradient_accumulation_steps=4):
    """
    Entraîne le modèle BERT et retourne l'historique d'entraînement
    
    Args:
        model: Modèle BERT à entraîner
        train_loader: DataLoader pour les données d'entraînement
        val_loader: DataLoader pour les données de validation
        test_loader: DataLoader pour les données de test
        device: Device sur lequel exécuter l'entraînement (cuda/cpu)
        epochs: Nombre d'époques d'entraînement
        gradient_accumulation_steps: Nombre de pas pour accumuler les gradients
    """
    # Paramètres d'optimisation
    optimizer = AdamW(model.parameters(), lr=2e-5, eps=1e-8)
    
    # Initialiser le scaler pour la précision mixte
    scaler = GradScaler()
    
    # Scheduler pour adapter le learning rate
    # Ajuster pour tenir compte de l'accumulation de gradient
    total_steps = (len(train_loader) // gradient_accumulation_steps) * epochs
    scheduler = get_linear_schedule_with_warmup(
        optimizer, 
        num_warmup_steps=0, 
        num_training_steps=total_steps
    )
    
    # Historique d'entraînement
    history = {
        'train_loss': [],
        'val_loss': [],
        'train_accuracy': [],
        'val_accuracy': []
    }
    
    # Boucle d'entraînement
    for epoch in range(epochs):
        # Mode d'entraînement
        model.train()
        train_loss = 0
        train_preds, train_true = [], []
        train_progress_bar = tqdm(train_loader, desc=f'Epoch {epoch+1}/{epochs} (Training)')
        
        # Reset des gradients au début de l'époque
        optimizer.zero_grad()
            
        for batch_idx, batch in enumerate(train_progress_bar):
            # Déplacer les tenseurs sur le device
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['label'].to(device)
            
            # Forward pass avec précision mixte
            with autocast():
                outputs = model(
                    input_ids=input_ids,
                    attention_mask=attention_mask,
                    labels=labels
                )
            
            # Calculer la perte et l'ajuster pour l'accumulation de gradient
            loss = outputs.loss / gradient_accumulation_steps
            train_loss += loss.item() * gradient_accumulation_steps  # Ajuster pour le reporting
            
            # Backward pass avec précision mixte
            scaler.scale(loss).backward()
            
            # Mise à jour des poids seulement à chaque étape d'accumulation
            if (batch_idx + 1) % gradient_accumulation_steps == 0:
                # Clip gradient norm
                scaler.unscale_(optimizer)
                torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
                
                # Optimizer step avec précision mixte
                scaler.step(optimizer)
                scheduler.step()
                scaler.update()
                optimizer.zero_grad()
            
            # Récupération des prédictions
            logits = outputs.logits
            preds = torch.argmax(logits, dim=1).cpu().numpy()
            true = labels.cpu().numpy()
            
            train_preds.extend(preds)
            train_true.extend(true)
            
            # Mise à jour de la barre de progression
            train_progress_bar.set_postfix({'training_loss': '{:.3f}'.format(loss.item())})
        
        # Calcul de l'accuracy d'entraînement
        train_accuracy = accuracy_score(train_true, train_preds)
        avg_train_loss = train_loss / len(train_loader)
        
        # Ajouter à l'historique
        history['train_loss'].append(avg_train_loss)
        history['train_accuracy'].append(train_accuracy)
        
        print(f"\nEpoch {epoch+1}/{epochs} - Average training loss: {avg_train_loss:.3f}")
        print(f"Epoch {epoch+1}/{epochs} - Training accuracy: {train_accuracy:.3f}")
        
        # Validation
        model.eval()
        val_loss = 0
        val_preds, val_true = [], []
        val_progress_bar = tqdm(val_loader, desc=f'Epoch {epoch+1}/{epochs} (Validation)')
        
        with torch.no_grad():
            for batch in val_progress_bar:
                input_ids = batch['input_ids'].to(device)
                attention_mask = batch['attention_mask'].to(device)
                labels = batch['label'].to(device)
                
                outputs = model(
                    input_ids=input_ids,
                    attention_mask=attention_mask,
                    labels=labels
                )
                
                loss = outputs.loss
                val_loss += loss.item()
                
                # Récupération des prédictions
                logits = outputs.logits
                preds = torch.argmax(logits, dim=1).cpu().numpy()
                true = labels.cpu().numpy()
                
                val_preds.extend(preds)
                val_true.extend(true)
                
                # Mise à jour de la barre de progression
                val_progress_bar.set_postfix({'validation_loss': '{:.3f}'.format(loss.item())})
        
        # Calcul de l'accuracy de validation
        val_accuracy = accuracy_score(val_true, val_preds)
        avg_val_loss = val_loss / len(val_loader)
        
        # Ajouter à l'historique
        history['val_loss'].append(avg_val_loss)
        history['val_accuracy'].append(val_accuracy)
        
        print(f"Epoch {epoch+1}/{epochs} - Validation loss: {avg_val_loss:.3f}")
        print(f"Epoch {epoch+1}/{epochs} - Validation accuracy: {val_accuracy:.3f}")
    
    # Évaluation finale sur le jeu de test
    test_metrics = evaluate_model(model, test_loader, device)
    
    return history, test_metrics


## Fonction d'évaluation

In [ ]:
# FONCTION D'ÉVALUATION
def evaluate_model(model, test_loader, device):
    """
    Évalue le modèle sur le jeu de test et retourne les métriques et prédictions
    """
    model.eval()
    test_preds, test_true = [], []
    test_probs = []
    
    with torch.no_grad():
        for batch in tqdm(test_loader, desc="Évaluation sur le jeu de test"):
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['label'].to(device)
            
            outputs = model(
                input_ids=input_ids,
                attention_mask=attention_mask
            )
            
            # Récupération des prédictions et probabilités
            logits = outputs.logits
            probs = torch.nn.functional.softmax(logits, dim=1)
            preds = torch.argmax(logits, dim=1).cpu().numpy()
            true = labels.cpu().numpy()
            
            test_preds.extend(preds)
            test_true.extend(true)
            test_probs.extend(probs[:, 1].cpu().numpy())  # Probabilité de la classe positive
    
    # Calcul des métriques
    accuracy = accuracy_score(test_true, test_preds)
    precision = precision_score(test_true, test_preds)
    recall = recall_score(test_true, test_preds)
    f1 = f1_score(test_true, test_preds)
    
    # Matrice de confusion
    cm = confusion_matrix(test_true, test_preds)
    
    # Classification report
    report = classification_report(test_true, test_preds, target_names=['Négatif', 'Positif'])
    
    # Courbe ROC et AUC
    fpr, tpr, _ = roc_curve(test_true, test_probs)
    roc_auc = auc(fpr, tpr)
    
    # Regrouper toutes les métriques
    metrics = {
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1': f1,
        'roc_auc': roc_auc,
        'fpr': fpr,
        'tpr': tpr,
        'confusion_matrix': cm,
        'classification_report': report,
        'predictions': test_preds,
        'true_labels': test_true,
        'probabilities': test_probs
    }
    
    return metrics

In [ ]:
# FONCTION POUR TRACER LA MATRICE DE CONFUSION
def plot_confusion_matrix(cm, class_names):
    """
    Tracer la matrice de confusion
    """
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=class_names, yticklabels=class_names)
    plt.xlabel('Prédictions')
    plt.ylabel('Valeurs réelles')
    plt.title('Matrice de confusion')
    return plt.gcf()

## Fonction pour tracer l'historique d'entraînement

In [ ]:
# FONCTION POUR TRACER L'HISTORIQUE D'ENTRAÎNEMENT
def plot_training_history(history, model_name, run_id=None):
    """
    Trace l'historique d'entraînement du modèle et l'enregistre dans MLflow
    """
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))
    
    # Tracer l'accuracy
    ax1.plot(history['train_accuracy'], label='train')
    ax1.plot(history['val_accuracy'], label='validation')
    ax1.set_title(f'Accuracy - {model_name}')
    ax1.set_xlabel('Epoch')
    ax1.set_ylabel('Accuracy')
    ax1.legend()
    
    # Tracer la loss
    ax2.plot(history['train_loss'], label='train')
    ax2.plot(history['val_loss'], label='validation')
    ax2.set_title(f'Loss - {model_name}')
    ax2.set_xlabel('Epoch')
    ax2.set_ylabel('Loss')
    ax2.legend()
    
    plt.tight_layout()
    
    # Sauvegarder la figure localement
    os.makedirs('content/bert-model', exist_ok=True)
    filename = f'content/bert-model/training_history_{model_name}.png'
    plt.savefig(filename)
    
    # Enregistrer dans MLflow en utilisant l'exécution existante
    if run_id:
        with mlflow.start_run(run_id=run_id):
            mlflow.log_figure(fig, f"training_history_{model_name}.png")
    
    return fig

## Fonction pour tracer la courbe ROC AUC

In [ ]:
# FONCTION POUR TRACER LA COURBE ROC
def plot_roc_curve(fpr, tpr, roc_auc, model_name, run_id=None):
    """
    Trace la courbe ROC et l'enregistre dans MLflow
    """
    plt.figure(figsize=(8, 6))
    plt.plot(fpr, tpr, label=f'ROC curve (area = {roc_auc:.3f})')
    plt.plot([0, 1], [0, 1], 'k--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('Taux de faux positifs')
    plt.ylabel('Taux de vrais positifs')
    plt.title(f'Courbe ROC - {model_name}')
    plt.legend(loc="lower right")
    
    # Sauvegarder la figure localement
    filename = f'content/bert-model/roc_curve_{model_name}.png'
    plt.savefig(filename)
    
    # Enregistrer dans MLflow en utilisant l'exécution existante
    if run_id:
        with mlflow.start_run(run_id=run_id):
            mlflow.log_figure(plt.gcf(), f"roc_curve_{model_name}.png")
    
    return plt.gcf()

## Enregistrement du modèle dans MLFlow

In [ ]:
# FONCTION POUR ENREGISTRER LE MODÈLE DANS MLFLOW
def log_model_to_mlflow(model, tokenizer, model_name, metrics, params=None):
    """
    Enregistre le modèle et ses performances dans MLflow
    """
    class_names = ['Négatif', 'Positif']
    
    with mlflow.start_run(run_name=f"Model_Bert_{model_name}"):
        # Enregistrer les paramètres du modèle
        if params:
            for key, value in params.items():
                mlflow.log_param(key, value)
        
        # Enregistrer les métriques de performance
        mlflow.log_metric("accuracy", metrics['accuracy'])
        mlflow.log_metric("precision", metrics['precision'])
        mlflow.log_metric("recall", metrics['recall'])
        mlflow.log_metric("f1", metrics['f1'])
        mlflow.log_metric("roc_auc", metrics['roc_auc'])
        
        # Tracer et enregistrer la matrice de confusion
        fig_cm = plot_confusion_matrix(metrics['confusion_matrix'], class_names)
        mlflow.log_figure(fig_cm, f"confusion_matrix_{model_name}.png")
        
        # Tracer et enregistrer la courbe ROC
        fig_roc = plot_roc_curve(metrics['fpr'], metrics['tpr'], metrics['roc_auc'], model_name)
        mlflow.log_figure(fig_roc, f"roc_curve_{model_name}.png")
        
        # Enregistrer le rapport de classification
        report_path = f"content/bert-model/classification_report.txt"
        os.makedirs(os.path.dirname(report_path), exist_ok=True)
        with open(report_path, "w") as f:
            f.write(metrics['classification_report'])
        mlflow.log_artifact(report_path)
        
        # Sauvegarder le modèle localement
        model_path = f"content/bert-model/model"
        os.makedirs(model_path, exist_ok=True)
        model.save_pretrained(model_path)
        tokenizer.save_pretrained(model_path)
        
        # Sauvegarder les métriques
        metrics_path = f"content/bert-model/metrics.json"
        with open(metrics_path, "w") as f:
            # Ne pas sauvegarder les arrays numpy car ils ne sont pas JSON serializable
            metrics_json = {
                k: v for k, v in metrics.items() 
                if k not in ['fpr', 'tpr', 'predictions', 'true_labels', 'probabilities', 'confusion_matrix']
            }
            # Pour les arrays numpy, convertir en listes
            metrics_json['confusion_matrix'] = metrics['confusion_matrix'].tolist()
            json.dump(metrics_json, f)
        mlflow.log_artifact(metrics_path)
        
        # Enregistrer le modèle dans MLflow
        mlflow.pytorch.log_model(
            model, 
            f"model_{model_name}",
            conda_env={
                'name': 'bert_env',
                'channels': ['defaults', 'pytorch', 'huggingface'],
                'dependencies': [
                    'python=3.8',
                    'pytorch',
                    'transformers'
                ]
            }
        )
        
        # Enregistrer le tokenizer comme artefact
        tokenizer_path = f"content/bert-model/tokenizer"
        os.makedirs(tokenizer_path, exist_ok=True)
        tokenizer.save_pretrained(tokenizer_path)
        mlflow.log_artifact(tokenizer_path, "tokenizer")
        
        return mlflow.active_run().info.run_id

## Entrainement du modèle

In [ ]:
# FONCTION PRINCIPALE D'ENTRAÎNEMENT
def train_bert_sentiment(data_path, model_name="DistilBERT-base", batch_size=4, epochs=3, sample_size=20000):
    """
    Fonction principale pour l'entraînement du modèle BERT
    """
    # Définir les paramètres
    params = {
        'model_name': model_name,
        'batch_size': batch_size,
        'learning_rate': 2e-5,
        'epochs': epochs,
        'max_length': 128,
        'sample_size': sample_size
    }
    
    # Charger les données
    print("Chargement du dataset...")
    column_names = ['target', 'ids', 'date', 'flag', 'user', 'text']
    raw_data = pd.read_csv(data_path, encoding='latin-1', names=column_names)
    
    # Préparer les données
    print("Préparation des données...")
    data_splits = prepare_data(raw_data, sample_size=sample_size)
    
    # Initialiser le tokenizer et le modèle
    print("Initialisation du modèle BERT...")
    # Utiliser DistilBERT qui est plus léger pour les GPUs avec mémoire limitée (comme GTX 1060 3GB)
    tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
    model = DistilBertForSequenceClassification.from_pretrained(
        'distilbert-base-uncased',
        num_labels=2
    )
    
    # Création des datasets et dataloaders
    # Utiliser un batch_size plus petit pour les GPUs avec mémoire limitée
    print("Préparation des dataloaders...")
    adjusted_batch_size = min(8, batch_size)  # Réduire le batch size pour GTX 1060 3GB
    print(f"Batch size ajusté à {adjusted_batch_size} pour économiser la mémoire GPU")
    
    train_dataset = TweetDataset(data_splits['train']['texts'], data_splits['train']['labels'], tokenizer)
    val_dataset = TweetDataset(data_splits['val']['texts'], data_splits['val']['labels'], tokenizer)
    test_dataset = TweetDataset(data_splits['test']['texts'], data_splits['test']['labels'], tokenizer)
    
    train_loader = DataLoader(train_dataset, batch_size=adjusted_batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=adjusted_batch_size * 2)
    test_loader = DataLoader(test_dataset, batch_size=adjusted_batch_size * 2)
    
    # Détection du device (GPU/CPU)
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f"Utilisation de: {device}")
    
    # Déplacement du modèle sur le device
    model = model.to(device)
    
    # Entraînement du modèle
    print("Début de l'entraînement...")
    # Ajouter gradient_accumulation_steps pour compenser la réduction du batch size
    gradient_accumulation_steps = 4
    print(f"Utilisation de l'accumulation de gradient avec {gradient_accumulation_steps} étapes")
    
    history, metrics = train_model(
        model, 
        train_loader, 
        val_loader, 
        test_loader, 
        device,
        epochs=epochs,
        gradient_accumulation_steps=gradient_accumulation_steps
    )
    
    # Enregistrer dans MLflow
    print("Enregistrement dans MLflow...")
    run_id = log_model_to_mlflow(model, tokenizer, model_name, metrics, params)
    
    # Tracer et enregistrer l'historique d'entraînement
    plot_training_history(history, model_name, run_id)
    
    # Retourner le modèle entrainé et le tokenizer
    return {
        'model': model,
        'tokenizer': tokenizer,
        'metrics': metrics,
        'run_id': run_id
    }

In [ ]:
# Entraîner le modèle
bert_pack = train_bert_sentiment(
    data_path=csv_file_path,
    model_name="DistilBERT-base",
    batch_size=16,
    epochs=3,
    sample_size=50000  # Utiliser un échantillon plus grand si vous avez assez de mémoire
)


## Chargement du modèle

In [ ]:
# FONCTION POUR CHARGER LE MODÈLE ENTRAINÉ
def load_bert_model(model_dir="content/bert-model/model"):
    """
    Charge le modèle BERT entraîné et le tokenizer
    """
    # Charger le modèle et le tokenizer
    model = DistilBertForSequenceClassification.from_pretrained(model_dir)
    tokenizer = DistilBertTokenizer.from_pretrained(model_dir)
    
    # Charger les métriques si disponibles
    metrics_path = f"content/bert-model/metrics.json"
    if os.path.exists(metrics_path):
        with open(metrics_path, "r") as f:
            metrics = json.load(f)
    else:
        metrics = None
    
    # Création d'un pack pour faciliter l'utilisation
    model_pack = {
        'model': model,
        'tokenizer': tokenizer,
        'metrics': metrics,
        'preprocess': preprocess_tweet_for_bert
    }
    
    return model_pack

## Fonction de prédiction

In [ ]:
# FONCTION DE PRÉDICTION
def predict_sentiment(tweet, model_pack):
    """
    Prédit le sentiment d'un tweet à l'aide du modèle BERT
    """
    model = model_pack['model']
    tokenizer = model_pack['tokenizer']
    preprocess = model_pack['preprocess']
    
    # Prétraitement du tweet
    processed_tweet = preprocess(tweet)
    
    # Tokenisation
    encoding = tokenizer(
        processed_tweet,
        return_tensors="pt",
        truncation=True,
        padding=True,
        max_length=128
    )
    
    # Détection du device
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = model.to(device)
    
    # Passer au mode évaluation
    model.eval()
    
    # Déplacer les tenseurs sur le device
    input_ids = encoding['input_ids'].to(device)
    attention_mask = encoding['attention_mask'].to(device)
    
    # Prédiction
    with torch.no_grad():
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        probabilities = torch.nn.functional.softmax(logits, dim=1)
    
    # Récupérer le sentiment prédit et le score de confiance
    predicted_class = torch.argmax(probabilities, dim=1).item()
    confidence_score = probabilities[0][predicted_class].item()
    raw_score = probabilities[0][1].item()  # Score de la classe positive
    
    # Convertir en étiquette lisible
    sentiment = "Positif" if predicted_class == 1 else "Négatif"
    
    # Résultat
    result = {
        'sentiment': sentiment,
        'confidence': confidence_score,
        'raw_score': raw_score
    }
    
    return result

## Test sur un ensemble de tweets

In [ ]:
# FONCTION DE TEST SUR ENSEMBLE DE TWEETS
def test_model_on_examples(model_pack, test_tweets):
    """
    Teste le modèle sur une liste de tweets et retourne un DataFrame avec les résultats des prédictions.
    """
    results = {
        "Positif": 0,
        "Négatif": 0
    }
    
    # Créer un DataFrame pour stocker les résultats
    predictions_data = []
    
    for i, tweet in enumerate(test_tweets):
        # Utiliser la fonction predict_sentiment avec model_pack
        result = predict_sentiment(tweet, model_pack)
        
        # Ajouter l'emoji approprié pour le sentiment
        emoji = "💚" if result['sentiment'] == "Positif" else "❌"
        
        # Ajouter les données au DataFrame
        predictions_data.append({
            "Tweet": tweet,
            "Sentiment": result['sentiment'],
            "Emoji": emoji,
            "Score de confiance": result['confidence'],
            "Score brut": result['raw_score'],
            "Index": i+1
        })
        
        # Comptage des résultats
        results[result['sentiment']] += 1
    
    # Créer le DataFrame
    df_predictions = pd.DataFrame(predictions_data)
    
    # Afficher un résumé
    print("\nTest du modèle sur les exemples:")
    print("="*50)
    print(f"Résumé: {len(test_tweets)} tweets analysés")
    print(f"Tweets positifs: {results['Positif']} ({(results['Positif']/len(test_tweets)*100):.1f}%)")
    print(f"Tweets négatifs: {results['Négatif']} ({(results['Négatif']/len(test_tweets)*100):.1f}%)")

    # Sauvegarder les résultats dans un fichier CSV
    os.makedirs('content/bert-model', exist_ok=True)
    df_predictions.to_csv("content/bert-model/predictions_results.csv", index=False)

    # Retourner le DataFrame
    return df_predictions

In [ ]:
# Liste améliorée de tweets pour tester le modèle
test_tweets_improved = [
    # Tweets positifs avec différentes caractéristiques
    "I absolutely love flying with @AirParadis! Their service is amazing and their staff is so helpful :) #bestairline #travel",
    "Just landed after a wonderful flight... The crew was totally awesome and the food was perfect!!! Can't wait to fly with them again :D",
    "OMG! You really have to try @AirParadis. Best. Flight. Ever. Their new seats are extremely comfortable. http://airparadis.com/newseats #travel #comfort",
    "We had a very pleasant experience with our flight today. The staff completely exceeded our expectations ;) #happy #AirParadis",
    "My first time flying business class and I'm absolutely amazed!!! The service is 100% worth it... @AirParadis never disappoints :)",
    
    # Tweets négatifs avec différentes caractéristiques
    "I hate how @AirParadis always delays their flights... This is the third time this month!!! :( #disappointed #travel",
    "The worst flight experience of my life? Definitely today with @AirParadis. Their customer service won't even respond to my complaints... #terrible",
    "Really @AirParadis??? Flight cancelled and no compensation??? That's how you treat your customers? SMH :(",
    "Our baggage was lost and nobody at @AirParadis seems to care! Don't they understand that we can't enjoy our vacation without our stuff??? http://badservice.com/complaint :-(",
    "Food was terrible, seats were uncomfortable, and the staff was not helpful at all... Never flying with @AirParadis again! #worstairline",
    
    # Tweets mixtes et nuancés
    "The flight was good but the food wasn't really what I expected... @AirParadis can do better! #mixedfeelings",
    "IDK what to think about my @AirParadis experience? The crew was nice but the flight was delayed for 2 hours... #confused",
    "My husband loved the flight but I didn't... Too bad they can't make everyone happy :-) #AirParadis",
    "TBH, @AirParadis has improved their service since last year! Not perfect yet, but they're trying... #progress",
    "We were so excited for our trip but then our flight got delayed... at least the staff was very apologetic and gave us free drinks! @AirParadis"
]

# Charger le modèle entraîné
model_pack = load_bert_model()

# Tester le modèle sur les exemples
df_results = test_model_on_examples(model_pack, test_tweets_improved)

print("Analyse de sentiment terminée! Résultats disponibles dans content/bert-model/predictions_results.csv")